## Connect to BigQuery

In [11]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Myrto.Setzi/Documents/Ironhack/code_alongs/ironhack_service_account_big_query.json"

In [6]:
import six
six.moves.reload_module(six)

<module 'six' from 'C:\\Users\\myrto.setzi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\six.py'>

In [7]:
from google.cloud import bigquery

In [12]:
client = bigquery.Client()

## Examples

Find the 10 most popular names and specify the gender

In [18]:
query_example='''
SELECT
  name,
  gender,
  SUM(number) AS total
FROM
  `bigquery-public-data.usa_names.usa_1910_2013`
WHERE
    year BETWEEN 2000 AND 2003 OR 
    (year>=2000 AND year<=2003) OR
    year IN (2000,2001,2002,2003)
GROUP BY
  1,2
HAVING 
    SUM(number)>4000000
ORDER BY
  total DESC
LIMIT 10
  '''

In [19]:
query_job = client.query(query=query_example)
df=query_job.to_dataframe()
df

name gender    total
0    James      M  4924235
1     John      M  4818746
2   Robert      M  4703680
3  Michael      M  4280040
4  William      M  3811998
5     Mary      F  3728041
6    David      M  3541625
7  Richard      M  2526927
8   Joseph      M  2467298
9  Charles      M  2237170

__Aggregate functions__

- Min 
- Max
- Count
- Sum 
- Avg 
- Countif

https://cloud.google.com/bigquery/docs/reference/standard-sql/aggregate_functions

__Having__ : where statement for agregate functions 

In [ ]:
query_example='''
SELECT
  name
  ,gender
  ,SUM(number) AS total
FROM
  `bigquery-public-data.usa_names.usa_1910_2013`
GROUP BY
  name
  ,gender
HAVING 
    SUM(number)>4000000
ORDER BY
  total DESC
LIMIT 10
  '''

![db_foto](./db_schema.png)

In [20]:
# Find the average price by type using the titles table where pub date is after 1991


In [5]:
# Find how many years each employee has been working in the company 

In [ ]:
# Find which employee started working there first

In [ ]:
# Count how many transactions within 1994 if payterms is net30

In [ ]:
# In which city live the most authors?

## Joins 

[![sql_joins](http://i.imgur.com/1m55Wqo.jpg)](http://i.imgur.com/1m55Wqo.jpg)

In [ ]:
# we know that publishers and titles have a one-to-many relationship. 
# Let's join them and get a count of the number of titles each publisher has published.


In [ ]:
query='''
SELECT 
    pubs.pub_name
    , COUNT(titles.title_id) AS titles
FROM 
     `ironhack-data-analytics-265219.publications.publishers` pubs
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titles`titles
ON 
    pubs.pub_id = titles.pub_id
GROUP BY 
    pubs.pub_name
'''

In [ ]:
query_job = client.query(query=query)
df=query_job.to_dataframe()
df

In [ ]:
# Because we used an INNER JOIN for this query, it returns only results for publishers whose pub_id is in both tables. What if we wanted the results to return records for all the publishers, regardless of whether they had published 
# any titles?

In [15]:
# What if we wanted to analyze how many units were sold for each title? We could declare our sales table first, our titles 
# table second, and use a RIGHT JOIN to ensure that our query returns a record for every title 
# (even the ones that did not have any sales).
query='''
SELECT 
    titles.title
    , titles.type
    , titles.price
    , SUM(sales.qty) AS units_sold
FROM 
    `ironhack-data-analytics-265219.publications.sales` sales
RIGHT JOIN 
     `ironhack-data-analytics-265219.publications.titles` titles
ON 
    sales.title_id = titles.title_id
WHERE
    titles.type="x"
GROUP BY 
    1,2,3
   '''

## Subqueries

In [ ]:
# Suppose we wanted to see sales by title for the two stores that averaged more than one item per order. 
# We can obtain these results by adding the stor_id field to our subquery, joining the subquery results to the sales table, 
# creating another join between the sales table and the title table, and then adding a WHERE clause to the main query so 
# that it returns only results 
# where the average items per order were greater than 1.

In [16]:
query='''
SELECT 
    store, 
    ord_num AS order_number, 
    ord_date AS order_date, 
    title AS title, 
    sales.qty AS qty, 
    price AS price, 
    type AS type
FROM (
    SELECT 
        stores.stor_id AS store_ID, 
        stores.stor_name AS store, 
        COUNT(DISTINCT(ord_num)) AS orders, 
        COUNT(title_id) AS items, 
        SUM(qty) AS qty
    FROM 
        `ironhack-data-analytics-265219.publications.sales` sales
    INNER JOIN 
        `ironhack-data-analytics-265219.publications.stores` stores 
    ON 
        stores.stor_id = sales.stor_id
    GROUP BY 
        1,2) summary
INNER JOIN 
    `ironhack-data-analytics-265219.publications.sales` sales 
ON 
    summary.store_ID = sales.stor_id
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titles` titles 
ON 
    sales.title_id = titles.title_id
WHERE 
    items / orders > 1
    '''

In [17]:
query_job = client.query(query=query)
df=query_job.to_dataframe()
df

store order_number  order_date  \
0                          News & Brews        P2121  1992-06-15   
1                          News & Brews        P2121  1992-06-15   
2                          News & Brews        P2121  1992-06-15   
3                          News & Brews        D4482  1994-09-14   
4  Doc-U-Mat: Quality Laundry and Books      N914008  1994-09-14   
5  Doc-U-Mat: Quality Laundry and Books      N914014  1994-09-14   
6  Doc-U-Mat: Quality Laundry and Books       P3087a  1993-05-29   
7  Doc-U-Mat: Quality Laundry and Books       P3087a  1993-05-29   
8  Doc-U-Mat: Quality Laundry and Books       P3087a  1993-05-29   
9  Doc-U-Mat: Quality Laundry and Books       P3087a  1993-05-29   

                                               title qty  price        type  
0          Fifty Years in Buckingham Palace Kitchens  20  11.95   trad_cook  
1  Onions, Leeks, and Garlic: Cooking Secrets of ...  40  20.95   trad_cook  
2                                     Sushi, Anyone?  20  14.99   trad_cook  
3                                Is Anger the Enemy?  10  10.95  psychology  
4                                Is Anger the Enemy?  20  10.95  psychology  
5                              The Gourmet Microwave  25   2.99    mod_cook  
6                Emotional Security: A New Algorithm  25   7.99  psychology  
7      Prolonged Data Deprivation: Four Case Studies  15  19.99  psychology  
8                                  Life Without Fear  25   7.00  psychology  
9  Computer Phobic AND Non-Phobic Individuals: Be...  20  21.59  psychology

__What are the most popular Citibike destinations for NYC couples?__

- Group Citibike trips together into ones that start and end at about the same time. 
- If the grouped-together trip had more than one person in it, then it’s counted as a group trip. 
- This could be a couple riding together, or a group of friends or family. 
- Possibly it could be people that happened to check out a bike at the same time and go to the same place at the same rate- considering that a rare occurrence. 
- Find which stations have the highest percentage of trips from groups versus single-rider trips.

https://medium.com/@TimSwast/what-are-the-most-popular-citibike-destinations-for-nyc-couples-1baf646fbba

In [ ]:
query_subquery='''
SELECT
  group_trips / (single_trips + group_trips) AS percent_groups,
  single_trips + group_trips AS total_trips,
  q.end_station_id AS end_station_id,
  stations.name AS name,
  stations.latitude AS latitude,
  stations.longitude AS longitude
FROM (
  SELECT
    COUNTIF(group_size = 1) AS single_trips,
    COUNTIF(group_size != 1) AS group_trips,
    end_station_id
  FROM (
    SELECT
      ROUND(UNIX_SECONDS(starttime) / 120) AS start,
      -- round to nearest 2 minutes
      ROUND(UNIX_SECONDS(stoptime) / 120) AS stop,
      -- round to nearest 2 minutes
      start_station_id,
      end_station_id,
      COUNT(*) AS group_size
    FROM
      `bigquery-public-data.new_york.citibike_trips`
    GROUP BY
      start,
      stop,
      start_station_id,
      end_station_id )
  GROUP BY
    end_station_id ) q
LEFT JOIN
  `bigquery-public-data.new_york.citibike_stations` AS stations
ON
  q.end_station_id = stations.station_id
ORDER BY
  percent_groups DESC
'''

In [ ]:
query_job = client.query(query=query_subquery)
df_subquery=query_job.to_dataframe()
df_subquery

## Action queries 

In [ ]:
CREATE TABLE publications.store_sales_summary AS
SELECT stores.stor_id AS StoreID, stores.stor_name AS Store, COUNT(DISTINCT(ord_num)) AS Orders, COUNT(title_id) AS Items, SUM(qty) AS Qty
FROM publications.sales sales
INNER JOIN publications.stores stores ON stores.stor_id = sales.stor_id
GROUP BY StoreID, Store;

In [ ]:
DELETE FROM publications.store_sales_summary
WHERE Qty < 80;

In [ ]:
INSERT INTO publications.store_sales_summary
SELECT stores.stor_id AS StoreID, stores.stor_name AS Store, COUNT(DISTINCT(ord_num)) AS Orders, COUNT(title_id) AS Items, SUM(qty) AS Qty
FROM publications.sales sales
INNER JOIN publications.stores stores ON stores.stor_id = sales.stor_id
GROUP BY StoreID, Store;

In [ ]:
UPDATE publications.store_sales_summary
SET Qty = Qty + 5